In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Eval")

LangSmith 추적을 시작합니다.
[프로젝트명]
Eval


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("outputs/synthetic_out.csv")
df

# Convert DataFrame to examples scheme
examples = [
    {
        "inputs": {"question": row["query"]},
        "outputs": {"answer": row["answers"]},
    }
    for _, row in df.iterrows()
]

print(f"Created {len(examples)} examples")
examples[:2]  # Show first 2 examples

Created 107 examples


[{'inputs': {'question': 'What are three factual statements about the publication responsibility, views, and territorial statements in the document?'},
  'outputs': {'answer': '1) The work is published under the responsibility of the Secretary-General of the OECD. 2) The opinions expressed do not necessarily reflect the official views of the OECD member countries. 3) The document and any data or map are without prejudice to the status of territories and to the delimitation of boundaries and to place names.'}},
 {'inputs': {'question': 'Which entity does Turkiye recognise as the Turkish Republic of Northern Cyprus (TRNC)?'},
  'outputs': {'answer': 'Turkiye recognises the Turkish Republic of Northern Cyprus (TRNC).'}}]

In [4]:
from langsmith import Client

client = Client()
dataset_name = "RAG eval dataset"

In [9]:
from langsmith import Client

client = Client()

# Create the dataset and examples in LangSmith
dataset_name = "RAG eval dataset"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['cdf372d6-397f-4bb4-ab9a-2cf52891a07d',
  '67f60b40-7e0a-49f9-817e-cecd5eea7f5c',
  '6ba5b5a7-a0dc-453f-ae3e-452bc765d20f',
  'ef816b23-87f7-4f5f-95bc-6b664175a8d7',
  'cd5bc95e-c111-43ef-a8a6-0be2c4902fa8',
  'd4fd1c86-b17f-43a5-9f52-bd3351d8c772',
  'f867c518-c265-4888-b759-8ad52c3b7199',
  '87c73137-0c9e-4cfc-83ab-fdc495d40451',
  '2d685e23-e793-4793-9300-1d48da3ec4f2',
  '4915398d-f0b7-4fcc-9c72-525b787c2726',
  '5b15abbd-3d10-429b-ae3d-bdf6874eabf4',
  'e5cf84f7-0300-4a2e-abc1-97bd78b98be1',
  'f3f1966f-bcee-4657-8680-6e4b622e69d2',
  '1edce126-609f-4dab-ac00-2e31e9128255',
  '92607d02-432a-457b-b058-721cee593e0a',
  '5b5e8f66-234a-441d-a47d-fafc3289fc8e',
  '439992a6-98ec-4256-aaf6-96d35559e3e4',
  'd38bc2d5-caf1-40ce-be77-ef799be96115',
  '10bc8ce6-0294-4e81-9045-312fc1bcf398',
  '4dd91ef9-4e90-43c5-8a66-b96cb46cec5d',
  '2406805c-49b5-40a7-b0a2-aa62ce23d0ae',
  '9406d909-817b-478b-88e3-400e8faec5bb',
  'be25a33c-787f-49c4-8cae-4c892bc68e80',
  'e4bedbcf-d22a-43

In [10]:
import pickle
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [18]:
with open("outputs/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [19]:
from typing import Annotated, TypedDict
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

vectorstore = FAISS.load_local(
    "faiss_index", 
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True  # needed in newer versions
)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 1
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

# GraphState 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "Question"]  # 질문
    context: Annotated[str, "Context"]  # 문서의 검색 결과
    answer: Annotated[str, "Answer"]  # 답변
    documents: Annotated[list[Document], "Documents"]  # 검색된 문서
    
# 문서 검색 노드
def retrieve_document(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = ensemble_retriever.invoke(latest_question)
    context = "".join([doc.page_content for doc in retrieved_docs])
    # 검색된 문서를 context 키에 저장합니다.
    return {"documents": retrieved_docs, "context": context}


# 답변 생성 노드
def llm_answer(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 검색된 문서를 상태에서 가져옵니다.
    context = state["context"]
   
    llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
      
    system_prompt = """You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        Answer in Korean.
    """
    
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Context: " + context},
        {"role": "user", "content": "Question: " + latest_question},
    ]
        
    # 체인을 호출하여 답변을 생성합니다.
    response = llm.invoke(prompt)
    print("response", response)
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return {"answer": response.content, "documents": state["documents"]}


# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

# 엣지 정의
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", END)  # 답변 -> 종료

# 그래프 진입점 설정
workflow.set_entry_point("retrieve")

# 체크포인터 설정
memory = MemorySaver()

# 컴파일
app = workflow.compile(checkpointer=memory)

In [20]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = GraphState(question="터키가 직면한 위기와 도전을 알려주세요.")

In [9]:
inputs

{'question': '터키가 직면한 위기와 도전을 알려주세요.'}

In [10]:
result = app.invoke(inputs, config)

response content='제공해주신 문맥(OECD 중간보고서와 터키의 주석)에는 터키에 대한 구체적·개별적 위기 목록은 없습니다. 다만 보고서에서 제시한 전반적 도전은 터키에도 적용될 수 있으므로, 그에 따라 터키가 직면할 수 있는 주요 쟁점을 정리하면 다음과 같습니다.\n\n- 무역·글로벌 협력: 세계무역체제 내에서 협력적으로 참여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화정책: 중앙은행은 경계를 유지해야 하고, 근원 인플레이션이 목표로 완만히 수렴할 경우 인플레이션 기대가 안정적이라는 전제 하에 정책금리를 낮출 수 있음.  \n- 재정건전성: 장기적 부채 지속가능성을 지키기 위한 재정 규율이 필요하며, 이를 통해 향후 충격에 대응할 여력을 확보해야 함.  \n- 구조개혁·생산성 제고: 생활수준을 지속적으로 끌어올리고 인공지능 등 신기술의 잠재력을 실현하려면 강화된 구조개혁이 필요함.  \n- 외교·지역문제: 문맥에 포함된 터키의 주석은 키프로스 문제(북키프로스 공화국 인정 등)에 대한 터키의 입장을 밝히고 있어, 지역 외교·정치적 긴장이 지속되는 점도 도전으로 볼 수 있음.\n\n보다 구체적이고 최신의 터키 경제·금융 위기(예: 초인플레이션, 환율 변동, 성장·실업 문제 등)에 대한 상세 정보를 원하시면 해당 주제에 대한 추가 자료를 불러오거나 일반적·공식 통계와 분석을 바탕으로 정리해 드리겠습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1057, 'prompt_tokens': 459, 'total_tokens': 1516, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens

In [12]:
result['answer'], result['documents']

('제공해주신 문맥(OECD 중간보고서와 터키의 주석)에는 터키에 대한 구체적·개별적 위기 목록은 없습니다. 다만 보고서에서 제시한 전반적 도전은 터키에도 적용될 수 있으므로, 그에 따라 터키가 직면할 수 있는 주요 쟁점을 정리하면 다음과 같습니다.\n\n- 무역·글로벌 협력: 세계무역체제 내에서 협력적으로 참여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화정책: 중앙은행은 경계를 유지해야 하고, 근원 인플레이션이 목표로 완만히 수렴할 경우 인플레이션 기대가 안정적이라는 전제 하에 정책금리를 낮출 수 있음.  \n- 재정건전성: 장기적 부채 지속가능성을 지키기 위한 재정 규율이 필요하며, 이를 통해 향후 충격에 대응할 여력을 확보해야 함.  \n- 구조개혁·생산성 제고: 생활수준을 지속적으로 끌어올리고 인공지능 등 신기술의 잠재력을 실현하려면 강화된 구조개혁이 필요함.  \n- 외교·지역문제: 문맥에 포함된 터키의 주석은 키프로스 문제(북키프로스 공화국 인정 등)에 대한 터키의 입장을 밝히고 있어, 지역 외교·정치적 긴장이 지속되는 점도 도전으로 볼 수 있음.\n\n보다 구체적이고 최신의 터키 경제·금융 위기(예: 초인플레이션, 환율 변동, 성장·실업 문제 등)에 대한 상세 정보를 원하시면 해당 주제에 대한 추가 자료를 불러오거나 일반적·공식 통계와 분석을 바탕으로 정리해 드리겠습니다.',
 [Document(metadata={'page': 24, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 108}, page_content='The Interim Report says that countries need to find ways of engaging co‑operatively within the global trading system  \nand working together to make trade policy more transparent

In [16]:
def rag_bot(question: str) -> dict:
    inputs = GraphState(question=question)
    res = app.invoke(inputs, config)
    return {"answer": res['answer'], "documents": res['documents']}

In [11]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. (2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=True
)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""
    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

In [12]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool, ..., "Provide the score on whether the answer addresses the question"
    ]

# Grade prompt
relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
relevance_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    RelevanceGrade, method="json_schema", strict=True
)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [13]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[
        bool, ..., "Provide the score on if the answer hallucinates from the documents"
    ]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. You will be given FACTS and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. (2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grounded_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    GroundedGrade, method="json_schema", strict=True
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

In [14]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool,
        ...,
        "True if the retrieved documents are relevant to the question, False otherwise",
    ]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a set of FACTS provided by the student. Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(
    model="gpt-5-nano", temperature=0
).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"
    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [21]:
res = rag_bot("터키가 직면한 위기와 도전을 알려주세요.")
res['answer'], res['documents']

response content='제공된 OECD 중간보고서(2025년 9월)에 따르면 문서에서 언급된 터키(Republic of Turkiye)가 직면한 주요 위기·도전은 다음과 같습니다.\n\n- 대외무역과 국제협력의 필요성: 세계 무역체제 내에서 다른 나라들과 협력적으로 관여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화 정책의 긴축·완화 판단: 근원 인플레이션이 목표로 완화될 것으로 예상될 때 인플레이션 기대가 잘 고정돼 있는 한 정책금리를 낮출 수 있으나, 중앙은행은 계속 경계를 늦추지 말아야 함(물가안정 리스크).  \n- 재정 지속가능성: 장기 채무 지속가능성을 지키고 향후 충격에 대응할 수 있도록 재정 규율을 유지할 필요가 있음.  \n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술에서 잠재력을 실현하기 위해 보다 강한 구조개혁 노력이 요구됨.  \n- 외교·영토 문제(사이프러스): 문서에는 사이프러스 관련 각주가 있어 터키가 북키프로스(TRNC)를 인정하고 있으며, 섬의 분단 문제와 관련한 외교적 입장·도전이 있음을 명시함.\n\n참고: 이 답변은 제공된 문서 내용에 근거한 요약입니다. 보다 구체적(예: 인플레이션율, 재정적자 규모, 실업률 등)인 경제지표나 최근 사건 기반 분석을 원하시면 추가 자료를 알려주시면 더 자세히 설명해 드리겠습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 966, 'prompt_tokens': 459, 'total_tokens': 1425, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_t

('제공된 OECD 중간보고서(2025년 9월)에 따르면 문서에서 언급된 터키(Republic of Turkiye)가 직면한 주요 위기·도전은 다음과 같습니다.\n\n- 대외무역과 국제협력의 필요성: 세계 무역체제 내에서 다른 나라들과 협력적으로 관여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화 정책의 긴축·완화 판단: 근원 인플레이션이 목표로 완화될 것으로 예상될 때 인플레이션 기대가 잘 고정돼 있는 한 정책금리를 낮출 수 있으나, 중앙은행은 계속 경계를 늦추지 말아야 함(물가안정 리스크).  \n- 재정 지속가능성: 장기 채무 지속가능성을 지키고 향후 충격에 대응할 수 있도록 재정 규율을 유지할 필요가 있음.  \n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술에서 잠재력을 실현하기 위해 보다 강한 구조개혁 노력이 요구됨.  \n- 외교·영토 문제(사이프러스): 문서에는 사이프러스 관련 각주가 있어 터키가 북키프로스(TRNC)를 인정하고 있으며, 섬의 분단 문제와 관련한 외교적 입장·도전이 있음을 명시함.\n\n참고: 이 답변은 제공된 문서 내용에 근거한 요약입니다. 보다 구체적(예: 인플레이션율, 재정적자 규모, 실업률 등)인 경제지표나 최근 사건 기반 분석을 원하시면 추가 자료를 알려주시면 더 자세히 설명해 드리겠습니다.',
 [Document(metadata={'page': 24, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 108}, page_content='The Interim Report says that countries need to find ways of engaging co‑operatively within the global trading system  \nand working together to make trade policy more transparent and predictable. C

In [21]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

examples = client.list_examples(dataset_name=dataset_name, limit=2)

experiment_results = client.evaluate(
    target,
    data=examples,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "langgraph, gpt-5-nano"},
)


View the evaluation results for experiment: 'rag-doc-relevance-4c7a9cd0' at:
https://smith.langchain.com/o/824afa80-3494-5b99-80b3-21be59fa9f24/datasets/69895610-d8c4-448d-9c65-1aa0f914c6c4/compare?selectedSessions=0d08b6f8-45b1-4c96-9a49-bbc12af233e4




0it [00:00, ?it/s]

response content='- 미국 성장궤적: 연간 GDP 성장률이 2024년 2.8%에서 2025년 1.8%, 2026년 1.5%로 둔화될 것으로 전망됨.  \n- 미국 둔화 이유: 효과적 관세율 상승의 본격적 영향, 정책 불확실성 고조, 순이민 감소 및 연방 인력 감축 등이 성장 둔화를 촉발(다만 첨단기술 분야의 기업투자 호조, 재정 지원, 추가 통화완화는 일부 상쇄 요인임).  \n- 캐나다 전망: 2025년 GDP 성장률 1.1%, 2026년 1.2%로 예상되며 대(對)미 수출에 대한 더 큰 무역제한이 성장 제약 요인으로 작용.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 516, 'prompt_tokens': 522, 'total_tokens': 1038, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CbJC2P3lXacfmxeLChxqzlAaOelE0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--0b662fbe-7aba-400a-b191-02b1f29cf0e4-0' usage_metadata={'input_tokens': 522, 'output_tokens': 516, 'total_tokens': 1038, 'input_token_details': {'audi

In [22]:
# Explore results locally as a dataframe if you have pandas installed
experiment_results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,Identify three key facts about growth projecti...,- 미국 성장궤적: 연간 GDP 성장률이 2024년 2.8%에서 2025년 1.8%...,[page_content='17. Aggregate consumer price in...,None,US GDP growth is projected to slow from 2.8% i...,True,True,True,True,9.351172,00438be7-0dc2-4fce-8b77-5531785c240f,4132d1bb-621b-41c0-ada5-c5ac8ed1dd85
1,What string appears on both lines of the content?,"문서의 두 줄 모두에 나타나는 문자열은 ""I 3""입니다. (대문자 I, 공백, 숫자 3)",[page_content='Note by the Republic of Turkiye...,None,I 5,False,True,True,True,8.762097,0125e6b3-b5e2-46d8-9756-18a85f738f09,ffad29f9-db8e-4dec-b343-96be41344f37
